Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---
<p style="padding: 10px; border: 2px solid red;">
    <b>Before you start:</b> Today is the day you should submit the dataset for your Unit 2 Build Week project. You can review the guidelines and make your submission in the Build Week course for your cohort on Canvas.</p>

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/main/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Module Project: Hyperparameter Tuning

This sprint, the module projects will focus on creating and improving a model for the Tanazania Water Pump dataset. Your goal is to create a model to predict whether a water pump is functional, non-functional, or needs repair.

Dataset source: [DrivenData.org](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/).

## Directions

The tasks for this project are as follows:

- **Task 1:** Use `wrangle` function to import training and test data.
- **Task 2:** Split training data into feature matrix `X` and target vector `y`.
- **Task 3:** Establish the baseline accuracy score for your dataset.
- **Task 4:** Build `clf_dt`.
- **Task 5:** Build `clf_rf`.
- **Task 6:** Evaluate classifiers using k-fold cross-validation.
- **Task 7:** Tune hyperparameters for best performing classifier.
- **Task 8:** Print out best score and params for model.
- **Task 9:** Create `submission.csv` and upload to Kaggle.

You should limit yourself to the following libraries for this project:

- `category_encoders`
- `matplotlib`
- `pandas`
- `pandas-profiling`
- `sklearn`

# I. Wrangle Data

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV


from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from pandas_profiling import ProfileReport

from sklearn.metrics import accuracy_score

In [48]:
def wrangle(fm_path, tv_path=None):
    if tv_path:
        df = pd.merge(pd.read_csv(fm_path, 
                                  na_values=[0, -2.000000e-08]),
                      pd.read_csv(tv_path)).set_index('id')
    else:
        df = pd.read_csv(fm_path, 
                         na_values=[0, -2.000000e-08],
                         index_col='id')

    # Drop constant columns
    df.drop(columns=['recorded_by'], inplace=True)

    # Drop HCCCs
    cutoff = 100
    drop_cols = [col for col in df.select_dtypes('object').columns
                 if df[col].nunique() > cutoff]
    df.drop(columns=drop_cols, inplace=True)

    # Drop duplicate columns
    dupe_cols = [col for col in df.head(15).T.duplicated().index
                 if df.head(15).T.duplicated()[col]]
    df.drop(columns=dupe_cols, inplace=True)     
    # Payment vs payment_type
    df = df.drop(columns='payment')
    # Waterpoint_type_group vs waterpoint_type
    df = df.drop(columns='waterpoint_type_group')
    # water_quality vs quality_group
    df = df.drop(columns='quality_group')
    # Source vs source_type
    df = df.drop(columns='source')
    # Extraction_type_class vs extraction_type_class
    df = df.drop(columns='extraction_type')

    # Drop the col with highest proportion of null values
    df = df.drop(columns='num_private')       

    return df

**Task 1:** Using the above `wrangle` function to read `train_features.csv` and `train_labels.csv` into the DataFrame `df`, and `test_features.csv` into the DataFrame `X_test`.

In [49]:
df = wrangle('train_features.csv','train_labels.csv')
X_test = wrangle('test_features.csv')

# II. Split Data

**Task 2:** Split your DataFrame `df` into a feature matrix `X` and the target vector `y`. You want to predict `'status_group'`.

**Note:** You won't need to do a train-test split because you'll use cross-validation instead.

In [50]:
X = df.drop(columns='status_group')
y = df['status_group']

# III. Establish Baseline

**Task 3:** Since this is a **classification** problem, you should establish a baseline accuracy score. Figure out what is the majority class in `y_train` and what percentage of your training observations it represents.

In [51]:
baseline_acc = y.value_counts(normalize=True).max()
print('Baseline Accuracy Score:', baseline_acc)

Baseline Accuracy Score: 0.5430899510092763


# IV. Build Models

**Task 4:** Build a `Pipeline` named `clf_dt`. Your `Pipeline` should include:

- an `OrdinalEncoder` transformer for categorical features.
- a `SimpleImputer` transformer fot missing values.
- a `DecisionTreeClassifier` Predictor.

**Note:** Do not train `clf_dt`. You'll do that in a subsequent task. 

In [52]:
clf_dt = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    DecisionTreeClassifier(random_state=42)
)

**Task 5:** Build a `Pipeline` named `clf_rf`. Your `Pipeline` should include:

- an `OrdinalEncoder` transformer for categorical features.
- a `SimpleImputer` transformer fot missing values.
- a `RandomForestClassifier` predictor.

**Note:** Do not train `clf_rf`. You'll do that in a subsequent task. 

In [53]:
clf_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, 
                            n_jobs=-1)
)

# V. Check Metrics

**Task 6:** Evaluate the performance of both of your classifiers using k-fold cross-validation.

In [54]:
cv_scores_dt = cross_val_score(clf_dt, X, y, cv=5, n_jobs=-1)
cv_scores_rf = cross_val_score(clf_rf, X, y, cv=5, n_jobs=-1)

In [55]:
print('CV scores DecisionTreeClassifier')
print(cv_scores_dt)
print('Mean CV accuracy score:', cv_scores_dt.mean())
print('STD CV accuracy score:', cv_scores_dt.std())

CV scores DecisionTreeClassifier
[0.75816498 0.74671717 0.75193603 0.75673401 0.74678003]
Mean CV accuracy score: 0.7520664441082827
STD CV accuracy score: 0.004807415029655781


In [56]:
print('CV score RandomForestClassifier')
print(cv_scores_rf)
print('Mean CV accuracy score:', cv_scores_rf.mean())
print('STD CV accuracy score:', cv_scores_rf.std())

CV score RandomForestClassifier
[0.80765993 0.80252525 0.80488215 0.8006734  0.80175099]
Mean CV accuracy score: 0.8034983459762483
STD CV accuracy score: 0.002498568132101792


# VI. Tune Model

**Task 7:** Choose the best performing of your two models and tune its hyperparameters using a `RandomizedSearchCV` named `model`. Make sure that you include cross-validation and that `n_iter` is set to at least `25`.

**Note:** If you're not sure which hyperparameters to tune, check the notes from today's guided project and the `sklearn` documentation. 

In [46]:
np.linspace(100, 700, 4)

array([100., 300., 500., 700.])

In [57]:
rf_n_estimators = [int(x) for x in np.linspace(100, 700, 4)]
rf_min_samples_split = [int(x) for x in np.linspace(6, 12, 7)]
rf_max_features = ['auto', 'sqrt', 'log2']
rf_max_depth = [int(x) for x in np.linspace(10, 50, 5)]
rf_criterion = ['gini', 'entropy']

rf_param = {'randomforestclassifier__n_estimators': rf_n_estimators,
            'randomforestclassifier__min_samples_split': rf_min_samples_split,
            'randomforestclassifier__max_features': rf_max_features,
            'randomforestclassifier__max_depth': rf_max_depth,
            'randomforestclassifier__criterion': rf_criterion
            }

In [58]:
model = RandomizedSearchCV(
    estimator=clf_rf, 
    param_distributions=rf_param,
    n_iter=25,
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=10
)
model.fit(X, y);

Fitting 5 folds for each of 25 candidates, totalling 125 fits


**Task 8:** Print out the best score and best params for `model`.

In [59]:
best_score = model.best_score_
best_params = model.best_params_

print('Best score for `model`:', best_score)
print('Best params for `model`:', best_params)

Best score for `model`: 0.811343581449651
Best params for `model`: {'randomforestclassifier__n_estimators': 700, 'randomforestclassifier__min_samples_split': 8, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_depth': 30, 'randomforestclassifier__criterion': 'entropy'}


In [60]:
model.best_score_

0.811343581449651

In [61]:
 model.best_params_

{'randomforestclassifier__n_estimators': 700,
 'randomforestclassifier__min_samples_split': 8,
 'randomforestclassifier__max_features': 'auto',
 'randomforestclassifier__max_depth': 30,
 'randomforestclassifier__criterion': 'entropy'}

In [63]:
y_pred = model.predict(X_test)

In [64]:
test = pd.read_csv('submission_04_13_rf.csv')
accuracy_score(test['status_group'], y_pred)

0.9508984538236523

# Communicate Results

**Task 9:** Create a DataFrame `submission` whose index is the same as `X_test` and that has one column `'status_group'` with your predictions. Next, save this DataFrame as a CSV file and upload your submissions to our competition site. 

**Note:** Check the `sample_submission.csv` file on the competition website to make sure your submissions follows the same formatting. 

In [66]:
submission = pd.DataFrame({'status_group':y_pred}, index=X_test.index)

In [68]:
submission.to_csv('submission_04_13_rf_3.csv')